In [1]:
import sys
print(sys.executable) 

c:\Users\SaisrisatyaPadala\AppData\Local\Programs\Python\Python312\python.exe


In [2]:
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import sqlite3
import pandas as pd

# Load cleaned dataset
df = pd.read_csv("Cleaned_Amazon_Sale_Report.csv")

# Create SQLite database file
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

print("SQLite Database Created Successfully!")


SQLite Database Created Successfully!


In [4]:
cursor.executescript("""

DROP TABLE IF EXISTS Customers;
DROP TABLE IF EXISTS Products;
DROP TABLE IF EXISTS Orders;
DROP TABLE IF EXISTS OrderDetails;

CREATE TABLE Customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    ship_city TEXT,
    ship_state TEXT,
    ship_country TEXT
);

CREATE TABLE Products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    SKU TEXT,
    Category TEXT,
    Size TEXT,
    ASIN TEXT
);

CREATE TABLE Orders (
    order_id TEXT PRIMARY KEY,
    order_date DATE,
    status TEXT,
    fulfilment TEXT,
    sales_channel TEXT,
    ship_service_level TEXT,
    courier_status TEXT,
    b2b BOOLEAN,
    fulfilled_by TEXT,
    customer_id INTEGER,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE OrderDetails (
    order_detail_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id TEXT,
    product_id INTEGER,
    qty INTEGER,
    amount REAL,
    promotion_ids TEXT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);

""")

conn.commit()
print("Normalized Tables Created Successfully!")


Normalized Tables Created Successfully!


In [17]:
df.rename(columns={
    'ship-city': 'ship_city',
    'ship-state': 'ship_state',
    'ship-country': 'ship_country',
    'ship-service-level': 'ship_service_level',
    'fulfilled-by': 'fulfilled_by',
    'promotion-ids': 'promotion_ids'
}, inplace=True)


In [6]:
# Insert distinct customer location info
customers_df = df[['ship_city', 'ship_state', 'ship_country']].drop_duplicates()

customers_df.to_sql("Customers", conn, if_exists="append", index=False)

# Map customer_id back into df
customer_map = pd.read_sql("SELECT * FROM Customers", conn)

df = df.merge(
    customer_map,
    on=['ship_city', 'ship_state', 'ship_country'],
    how='left'
)

print("Customers inserted & customer_id mapped successfully!")


Customers inserted & customer_id mapped successfully!


In [7]:
# Insert products table data
products_df = df[['SKU', 'Category', 'Size', 'ASIN']].drop_duplicates()

products_df.to_sql("Products", conn, if_exists="append", index=False)

# Fetch mapping from SQLite
product_map = pd.read_sql("SELECT * FROM Products", conn)

# Map product_id into main df
df = df.merge(
    product_map,
    on=['SKU', 'Category', 'Size', 'ASIN'],
    how='left'
)

print("Products inserted & product_id mapped successfully!")


Products inserted & product_id mapped successfully!


In [13]:
cursor.execute("DELETE FROM OrderDetails;")
cursor.execute("DELETE FROM Orders;");
conn.commit()
print("Orders and OrderDetails reset successful!")


Orders and OrderDetails reset successful!


In [14]:
orders_df = df[['Order ID','Date','Status','Fulfilment',
                'Sales Channel','ship_service_level',
                'Courier Status','B2B','fulfilled_by','customer_id']].drop_duplicates(subset=['Order ID'])

orders_df.rename(columns={
    'Order ID':'order_id',
    'Date':'order_date',
    'Status':'status',
    'Fulfilment':'fulfilment',
    'Sales Channel':'sales_channel',
    'Courier Status':'courier_status'
}, inplace=True)

orders_df.to_sql("Orders", conn, if_exists="append", index=False)

print("Orders inserted successfully! 🚀")


Orders inserted successfully! 🚀


In [18]:
order_details_df = df[['Order ID','product_id','Qty','Amount','promotion_ids']]

order_details_df.rename(columns={
    'Order ID':'order_id',
    'Qty':'qty',
    'Amount':'amount'
}, inplace=True)

order_details_df.to_sql("OrderDetails", conn, if_exists="append", index=False)
conn.commit()

print("OrderDetails inserted successfully! 🎉")


C:\Users\SaisrisatyaPadala\AppData\Local\Temp\ipykernel_22000\4105014512.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_details_df.rename(columns={


OrderDetails inserted successfully! 🎉


In [19]:
print(pd.read_sql("SELECT * FROM Customers LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM Products LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM Orders LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM OrderDetails LIMIT 5;", conn))


   customer_id    ship_city   ship_state ship_country
0            1       Mumbai  Maharashtra           IN
1            2    Bengaluru    Karnataka           IN
2            3  Navi Mumbai  Maharashtra           IN
3            4   Puducherry   Puducherry           IN
4            5      Chennai   Tamil Nadu           IN
   product_id              SKU       Category Size        ASIN
0           1   SET389-KR-NP-S            Set    S  B09KXVBD7Z
1           2  JNE3781-KR-XXXL          Kurta  3XL  B09K3WFS32
2           3    JNE3371-KR-XL          Kurta   XL  B07WV4JV4D
3           4       J0341-DR-L  Western Dress    L  B099NRCT7B
4           5  JNE3671-TU-XXXL            Top  3XL  B098714BZP
              order_id  order_date                        status fulfilment  \
0  405-8078784-5731545  2022-04-30                     Cancelled   Merchant   
1  171-9198151-1101146  2022-04-30  Shipped - Delivered to Buyer   Merchant   
2  404-0687676-7273146  2022-04-30                       Ship

In [21]:
 
print(pd.read_sql("SELECT COUNT(*) FROM OrderDetails;", conn))
 
pd.read_sql("SELECT * FROM OrderDetails LIMIT 10;", conn)


   COUNT(*)
0    128969


,order_detail_id,order_id,product_id,qty,amount,promotion_ids
0,1,405-8078784-5731545,1,0,0.0,No Promotion
1,2,171-9198151-1101146,2,1,406.0,Amazon PLCC Free-Financing Universal Merchant ...
2,3,404-0687676-7273146,3,1,329.0,IN Core Free Shipping 2015/04/08 23-48-5-108
3,4,403-9615377-8133951,4,0,0.0,No Promotion
4,5,407-1069790-7240320,5,1,574.0,No Promotion
5,6,404-1490984-4578765,6,1,824.0,IN Core Free Shipping 2015/04/08 23-48-5-108
6,7,408-5748499-6859555,7,1,653.0,IN Core Free Shipping 2015/04/08 23-48-5-108
7,8,406-7807733-3785945,8,1,399.0,Amazon PLCC Free-Financing Universal Merchant ...
8,9,407-5443024-5233168,9,0,0.0,IN Core Free Shipping 2015/04/08 23-48-5-108
9,10,402-4393761-0311520,10,1,363.0,No Promotion


In [22]:
print(pd.read_sql("SELECT COUNT(*) FROM Orders;", conn)) 

pd.read_sql("SELECT * FROM Orders LIMIT 10;", conn) 


   COUNT(*)
0    120378


,order_id,order_date,status,fulfilment,sales_channel,ship_service_level,courier_status,b2b,fulfilled_by,customer_id
0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,Pending,0,Merchant,1
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,Shipped,0,Merchant,2
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,Shipped,1,Amazon,3
3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,Pending,0,Merchant,4
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,Shipped,0,Amazon,5
5,404-1490984-4578765,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,Shipped,0,Amazon,6
6,408-5748499-6859555,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,Shipped,0,Amazon,7
7,406-7807733-3785945,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,Shipped,0,Merchant,8
8,407-5443024-5233168,2022-04-30,Cancelled,Amazon,Amazon.in,Expedited,Cancelled,0,Amazon,8
9,402-4393761-0311520,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,Shipped,0,Amazon,5


In [ ]:
##### Analytical SQL Queries

In [ ]:
#Top-Selling Product by Month

In [25]:
query1 = """
SELECT 
    strftime('%m', o.order_date) AS month,
    p.Category,
    SUM(od.qty) AS total_quantity
FROM OrderDetails od
JOIN Orders o ON od.order_id = o.order_id
JOIN Products p ON od.product_id = p.product_id
GROUP BY month, p.Category
ORDER BY total_quantity DESC
LIMIT 10;
"""

pd.read_sql(query1, conn)


,month,Category,total_quantity
0,04,Set,18143
1,04,Kurta,17676
2,05,Set,14405
3,06,Kurta,13748
4,05,Kurta,13549
5,06,Set,12671
6,05,Western Dress,5408
7,06,Western Dress,4751
8,04,Western Dress,3778
9,05,Top,3764


In [ ]:
#Sales by Region

In [26]:
query2 = """
SELECT 
    c.ship_state AS region,
    SUM(od.amount) AS total_sales
FROM OrderDetails od
JOIN Orders o ON od.order_id = o.order_id
JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY region
ORDER BY total_sales DESC;
"""

pd.read_sql(query2, conn)


,region,total_sales
0,Maharashtra,12223831.0
1,Karnataka,9649981.0
2,Telangana,6290128.0
3,Uttar Pradesh,6186123.0
4,Tamil Nadu,5954349.0
5,Delhi,4007579.0
6,Kerala,3377321.0
7,West Bengal,3207213.0
8,Andhra Pradesh,2882206.0
9,Haryana,2654274.0


In [ ]:
#High-Spend Customers (> 5000 INR)

In [27]:
query3 = """
SELECT 
    c.customer_id,
    c.ship_city,
    SUM(od.amount) AS total_spend
FROM Customers c
JOIN Orders o ON c.customer_id = o.customer_id
JOIN OrderDetails od ON od.order_id = o.order_id
GROUP BY c.customer_id
HAVING total_spend > 5000
ORDER BY total_spend DESC;
"""

pd.read_sql(query3, conn)


,customer_id,ship_city,total_spend
0,2,Bengaluru,6729194.0
1,8,Hyderabad,5063709.0
2,1,Mumbai,3954578.0
3,13,New Delhi,3620190.0
4,5,Chennai,3296242.0
...,...,...,...
1010,3927,Bhadohi,5027.0
1011,681,Nasik,5024.0
1012,3645,Bhowali,5014.0
1013,723,Butibori,5013.0
